# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [4]:
def expected_return(P, rf, mkt_rp):
    s1 = P.loc[0,"Amount Invested"] / P["Amount Invested"].sum()
    s2 = 1 - s1
    b = s1*P.loc[0,"Beta"] + s2*P.loc[1,"Beta"]
    er = rf + b*mkt_rp
    return er

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [7]:
def volatility(P, cor):
    s1 = P.loc[0,"Amount Invested"] / P["Amount Invested"].sum()
    s2 = 1 - s1
    v1 = P.loc[0,"Volatility"]
    v2 = P.loc[1,"Volatility"]
    var = (s1*v1)**2 + (s2*v2)**2 + 2*s1*v1*s2*v2*cor
    return (var)**(1/2)

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [9]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    x = vol / mkt_vol
    er2 = rf+mkt_rp*x
    return er2

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [100]:
def CAPM_beta(return_path, factors_path):
    ret = pd.read_csv(return_path)
    fac = pd.read_csv(factors_path)
    fac["Mkt-RF"] = fac["Mkt-RF"] / 100
    fac["RF"] = fac["RF"] / 100

    # Format the date in the return data
    ret["date1"] = ret["year"].astype(str) + ret["month"].astype(str).str.zfill(2)

    # Convert the "date" column in factors data to object type
    fac["date"] = fac["date"].astype(str)

    # Merge the return and factors data on date
    merged = pd.merge(ret, fac, left_on=['date1'], right_on=['date'])

    # Calculate excess returns
    merged['excess_return'] = merged['RET'] - merged['RF']

    # Calculate market excess return
    merged['mkt_excess_return'] = merged['Mkt-RF']

    # Calculate Cov(R_i-Rf, R_mkt – Rf) for each CUSIP
    output_beta = merged.groupby('CUSIP').apply(lambda group: pd.Series({
        'BETA': (group['excess_return']).cov(group['Mkt-RF']) / group['Mkt-RF'].var()
    })).reset_index()

    # Calculate the expected return for each CUSIP
    output_ret = pd.DataFrame(data={
        'EXP_RETURN': merged.groupby('CUSIP')['RET'].mean()
    }).reset_index()

    # Merge the DataFrames on 'CUSIP'
    final = pd.merge(output_beta, output_ret, on='CUSIP')
    return final

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [101]:
def slope(data):
    c = np.polyfit(data['BETA'], data['EXP_RETURN'], 1)
    slope = c[1]
    # This should return 0.007981588120535642... with the provided data.
    return slope